In [37]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from gbor.main import BoostedOrdinal
from sklearn.tree import DecisionTreeRegressor
from sklearn.model_selection import GridSearchCV
import time
from sklearn.model_selection import cross_val_score, RepeatedKFold

#wine_red = pd.read_csv('../data/winequality-red.csv', sep = ';')
wine_red = pd.read_csv('../data/winequality-white.csv', sep = ';')
wine_red['quality'] = wine_red['quality'] - np.min(wine_red['quality'])

X, y = wine_red.drop(columns = ['quality']).to_numpy(), wine_red['quality'].to_numpy(dtype = 'int')
#X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.4, stratify = y)

cv = RepeatedKFold(n_repeats = 10, n_splits = 5)

In [38]:
new_gbor = BoostedOrdinal(
    n_iter_no_change = 10, max_iter = 1000
    , base_learner = DecisionTreeRegressor(max_depth = 6)
    , lr_g = 1e-1
    , lr_theta = 5#e-4
    , validation_stratify = False
    , validation_fraction = 0.2
)
start_time = time.time()
my_cv = cross_val_score(
    new_gbor, X, y, cv = cv
    , scoring = 'accuracy'
    , n_jobs = -1
)
end_time = time.time()
print('time: {} sec'.format(round(end_time - start_time, 1)))

print(my_cv)

time: 14.4 sec
[0.55102041 0.5744898  0.5377551  0.55566905 0.60163432 0.51530612
 0.5744898  0.58469388 0.58222676 0.56792646 0.56836735 0.56122449
 0.56836735 0.56792646 0.56384065 0.56938776 0.57755102 0.56428571
 0.55158325 0.56690501 0.5744898  0.57244898 0.59183673 0.5566905
 0.54034729 0.56836735 0.53367347 0.57244898 0.52298264 0.54749745
 0.58061224 0.56122449 0.54693878 0.54239019 0.5607763  0.5622449
 0.56836735 0.52142857 0.55158325 0.60265577 0.57857143 0.56836735
 0.54489796 0.58426966 0.53421859 0.56428571 0.58469388 0.56530612
 0.54954035 0.57711951]


In [39]:
my_cv.mean()

0.5626985261929082

In [21]:
parameter_grid2 = {'base_learner__max_depth': [3, 6, 9], 'lr_g': [1e-1, 1e-2, 1e-3]}
new_gridsearch = GridSearchCV(
    new_gbor
    , parameter_grid2
    , scoring = 'accuracy'
    , n_jobs = -1
)
start_time = time.time()
my_cv2 = cross_val_score(
    new_gridsearch, X, y, cv = cv
    , scoring = 'accuracy'
    , n_jobs = -1
    , verbose = 1
)
end_time = time.time()
print('time: {} sec'.format(round(end_time - start_time, 1)))

print(my_cv2)

[Parallel(n_jobs=-1)]: Using backend LokyBackend with 12 concurrent workers.


KeyboardInterrupt: 

In [ ]:
from sklearn.model_selection import RandomizedSearchCV
from scipy.stats import loguniform

parameter_distros = {
    'base_learner__max_depth': [3, 4, 5, 6, 7, 8, 9]
    , 'lr_g': loguniform(1e-3, 1e-1)
}
new_randomsearch = RandomizedSearchCV(
    new_gbor
    , parameter_distros
    , scoring = 'accuracy'
    , n_iter = 10
)
start_time = time.time()
my_cv3 = cross_val_score(
    new_randomsearch, X, y, cv = cv
    , scoring = 'accuracy'
    , n_jobs = -1
    , verbose = 1
)
end_time = time.time()
print('time: {} sec'.format(round(end_time - start_time, 1)))

print(my_cv3)

In [ ]:
#from scipy import stats

# Perform paired t-test
#t_stat, p_value = stats.ttest_rel(my_cv, my_cv2)
#print(f"Paired t-test: t-statistic = {t_stat}, p-value = {p_value}")


In [40]:
from mord import LogisticIT
mord_model = LogisticIT(alpha = 0.0)

start_time = time.time()
my_cv4 = cross_val_score(
    mord_model, X, y, cv = cv
    , scoring = 'accuracy'
    , n_jobs = -1
    , verbose = 1
)
end_time = time.time()
print('time: {} sec'.format(round(end_time - start_time, 1)))

print(my_cv4)

[Parallel(n_jobs=-1)]: Using backend LokyBackend with 12 concurrent workers.
[Parallel(n_jobs=-1)]: Done  26 tasks      | elapsed:   27.4s


time: 40.7 sec
[0.5255102  0.53061224 0.53469388 0.52808989 0.50459653 0.49897959
 0.53163265 0.51836735 0.53728294 0.52706844 0.52346939 0.53265306
 0.53265306 0.53421859 0.49540347 0.53163265 0.50204082 0.52959184
 0.56384065 0.51174668 0.51938776 0.49183673 0.55510204 0.52604699
 0.53421859 0.52040816 0.56428571 0.51734694 0.50970378 0.50663943
 0.50510204 0.54387755 0.52959184 0.50663943 0.53524004 0.50510204
 0.54489796 0.53571429 0.51174668 0.53319714 0.52142857 0.52346939
 0.52040816 0.52706844 0.53319714 0.5622449  0.50408163 0.51020408
 0.50766088 0.53115424]


[Parallel(n_jobs=-1)]: Done  50 out of  50 | elapsed:   40.6s finished


In [41]:
my_cv4.mean(), my_cv.mean()#, my_cv2.mean(), my_cv3.mean()

(0.5246217297950845, 0.5626985261929082)

In [42]:
from scipy import stats

t_stat, p_value = stats.ttest_rel(my_cv4, my_cv)
print(f"Paired t-test: t-statistic = {t_stat}, p-value = {p_value}")


Paired t-test: t-statistic = -10.356507668275249, p-value = 6.212156264671022e-14


In [ ]:
#t_stat, p_value = stats.ttest_rel(my_cv2, my_cv3)
#print(f"Paired t-test: t-statistic = {t_stat}, p-value = {p_value}")


In [ ]:
np.unique(y, return_counts=True)